# Imports

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import pandas as pd

# Import Functions
import config
import io_read_write
import wrangle
import columns
import helpers

# Import Data Required
df_nursing_services = io_read_write.read_xlsx("./MoH_Model_Input.xlsx", sheet_name="Nursing Services")
df_benchmarks = io_read_write.read_xlsx("./MoH_Model_Input.xlsx", sheet_name="Benchmarks")
df_health_clusters = io_read_write.read_xlsx("./MoH_Model_Input.xlsx", sheet_name="Health Clusters")
df_demand_data_input = io_read_write.read_xlsx("./MoH_Model_Input.xlsx", sheet_name="Demand Data by Speciality")
df_scenario_criteria = io_read_write.read_xlsx("./MoH_Model_Input.xlsx", sheet_name="Scenarios")

# Melt Relevant Datasets
df_demand_data = df_demand_data_input.melt(
    id_vars=["Patient Care Area", "Nursing Service", "Driver"],
    var_name="Cluster",
    value_name="driver_value"
)

# Normalize Column Names
df_nursing_services = wrangle.normalize_column_names(df_nursing_services)
df_benchmarks = wrangle.normalize_column_names(df_benchmarks)
df_health_clusters = wrangle.normalize_column_names(df_health_clusters)
df_demand_data = wrangle.normalize_column_names(df_demand_data)
df_scenario_criteria = wrangle.normalize_column_names(df_scenario_criteria)

# Generate Benchmarks

In [56]:
df_benchmarks_stats = (
    df_benchmarks
    .groupby([columns.PATIENT_CARE_AREA, columns.NURSING_SERVICE, columns.DRIVER], dropna=False)
    .apply(helpers.calc_stats, value_col=columns.RATIO_VALUE, include_groups=False)
    .reset_index()
)

df_avg_ratio_by_area_country = (
    df_benchmarks
    .groupby([columns.COUNTRY, columns.PATIENT_CARE_AREA], dropna=False)
    [columns.RATIO_VALUE]
    .mean()
    .reset_index()
    .rename(columns={columns.RATIO_VALUE: "average_ratio_value"})
)

df_avg_overall_ratio_by_country = (
    df_benchmarks
    [df_benchmarks[columns.PATIENT_CARE_AREA] == "Overall"]
    .groupby([columns.COUNTRY], dropna=False)
    [columns.RATIO_VALUE]
    .mean()
    .reset_index()
    .rename(columns={columns.RATIO_VALUE: "average_ratio_value"})
)

# Calculate Demand Data by Cluster